In [1]:
import torch
import torch.optim as optim
import os
from resolve.utilities import utilities as utils
from resolve.helpers import DataGeneration
from resolve.helpers import Trainer, ModelsManager
from resolve.helpers import AsymmetricFocalWithFPPenalty, log_prob, bce_with_logits, recon_loss_mse
from torch.utils.tensorboard import SummaryWriter
import yaml
import json


In [2]:
# Set the path to the yaml settings file here
path_to_settings = "./binary-black-hole/"
with open(f"{path_to_settings}/settings.yaml", "r") as f:
    config_file = yaml.safe_load(f)

torch.manual_seed(0)
version = config_file["path_settings"]["version"]
path_out = f'{config_file["path_settings"]["path_out_model"]}/model-{version}'

In [3]:
model_name = config_file["model_settings"]["network"]["model_used"]
network_config = config_file["model_settings"]["network"]["models"][model_name]
network_config["d_y"] = utils.get_feature_and_label_size(config_file)[1]
network_config["d_theta"]  = len(config_file["simulation_settings"]["theta_labels"])
network_config["d_phi"] = len(config_file["simulation_settings"]["phi_labels"])

manager = ModelsManager(network_config)
model = manager.get_network(config_file["model_settings"]["network"]["model_used"])

encoder 15 32
encoder 32 64
encoder 64 48
encoder 48 32
decoder 46 48
decoder 48 64
decoder 64 48
decoder 48 32
decoder 32 2


In [4]:
# load data:
dataset_train = DataGeneration(mode = "train", 
                                config_file=config_file
                                )

dataset_train.set_dataset()

if config_file["model_settings"]["train"]["dataset"]["use_feature_normalization"] == "zscore":
    print("theta mean: ", dataset_train.dataset._normalizer._get_scaler("theta").mean_)
    print("phi mean: ", dataset_train.dataset._normalizer._get_scaler("phi").mean_)
elif config_file["model_settings"]["train"]["dataset"]["use_feature_normalization"] == "minmax":
    print("theta mean: ", dataset_train.dataset._normalizer._get_scaler("theta").data_range_)
    print("phi mean: ", dataset_train.dataset._normalizer._get_scaler("phi").data_range_)



positives ratio  tensor(0.0073)
applying mixup
theta mean:  [1.50423200e-02 4.99557795e+00 5.00162000e+02 4.98470000e+02]
phi mean:  [3.09912000e-02 1.38451402e+01 7.49881529e+01 3.83905393e+01
 1.93798800e-01 5.03367206e-01 1.08481965e+02 1.17347808e+01
 1.09972412e+01 9.90653600e-01]


In [5]:
os.system(f'mkdir -p {path_out}/model_{version}_tensorboard_logs')
os.system(f'rm {path_out}/model_{version}_tensorboard_logs/events*')
writer = SummaryWriter(log_dir=f'{path_out}/model_{version}_tensorboard_logs')

In [6]:
config_file["model_settings"]["train"]["dataset"]["positive_ratio_train"]

In [7]:
%%time
optimizer = optim.Adam(model.parameters(), lr=config_file["model_settings"]["train"]["learning_rate"])

# Instantiate the training wrapper for the first phase
trainer = Trainer(model, dataset_train)

trainer.epochs = config_file["model_settings"]["train"]["training_epochs"]

if utils.get_nested(config_file, ["model_settings","train","dataset","use_schedule"], False) == True:
        trainer.criterion = AsymmetricFocalWithFPPenalty(
                        alpha_pos=1.,
                        alpha_neg=1.,
                        gamma_pos=2.,
                        gamma_neg=2.,
                        lambda_fp=0.,
                        tau_fp=0.5,
                        lambda_tp=0.,
                        tau_tp=0.5,
                        reduction=utils.get_nested(config_file, ["model_settings","train","loss","reduction"], "mean"),
                        base_loss_fn=globals()[utils.get_nested(config_file, ["model_settings","train","loss","base_loss_fn"], "bce_with_logits")],
                )

        if utils.get_nested(config_file, ["model_settings","train","dataset","skip_warmup"], False) == True:
                print("loading warm up")
                model.load_state_dict(torch.load(f'{path_out}/model_{version}_warmup_model.pth'))
        else:
                # Train the model
                summary_train = trainer.warm_up(target_pos_frac = utils.get_nested(config_file, ["model_settings","train","dataset","positive_ratio_train"], None),
                        optimizer= optimizer,
                        writer=writer,
                        monitor = "pr_auc",
                        mode = "max",
                        save_best = True,
                        patience = 5,
                        max_epochs_per_phase = 2)

                torch.save(model.state_dict(), f'{path_out}/model_{version}_warmup_model.pth')


CPU times: user 931 ms, sys: 407 ms, total: 1.34 s
Wall time: 319 ms


In [8]:
model

ConditionalNeuralProcess(
  (_encoder): DeterministicEncoder(
    (encoder): Sequential(
      (0): Linear(in_features=15, out_features=32, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=32, out_features=64, bias=True)
      (3): ReLU(inplace=True)
      (4): Linear(in_features=64, out_features=48, bias=True)
      (5): ReLU(inplace=True)
      (6): Linear(in_features=48, out_features=32, bias=True)
    )
  )
  (_decoder): DeterministicDecoder(
    (decoder): Sequential(
      (0): Linear(in_features=46, out_features=48, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=48, out_features=64, bias=True)
      (3): ReLU(inplace=True)
      (4): Linear(in_features=64, out_features=48, bias=True)
      (5): ReLU(inplace=True)
      (6): Linear(in_features=48, out_features=32, bias=True)
      (7): ReLU(inplace=True)
      (8): Linear(in_features=32, out_features=2, bias=True)
    )
  )
)

In [9]:
%%time

trainer.criterion = AsymmetricFocalWithFPPenalty(
                alpha_pos=utils.get_nested(config_file, ["model_settings","train","loss","alpha_pos"], 1.),
                alpha_neg=utils.get_nested(config_file, ["model_settings","train","loss","alpha_neg"], 1.),
                gamma_pos=utils.get_nested(config_file, ["model_settings","train","loss","gamma_pos"], 0.),
                gamma_neg=utils.get_nested(config_file, ["model_settings","train","loss","gamma_neg"], 0.),
                lambda_fp=utils.get_nested(config_file, ["model_settings","train","loss","lambda_fp"],0.),
                tau_fp=utils.get_nested(config_file, ["model_settings","train","loss","tau_fp"],0.5),
                lambda_tp=utils.get_nested(config_file, ["model_settings","train","loss","lambda_tp"],0.),
                tau_tp=utils.get_nested(config_file, ["model_settings","train","loss","tau_tp"],0.5),
                reduction=utils.get_nested(config_file, ["model_settings","train","loss","reduction"], "mean"),
                base_loss_fn=globals()[utils.get_nested(config_file, ["model_settings","train","loss","base_loss_fn"], "bce_with_logits")],
            )

# Train the model
summary_train = trainer.fit(optimizer=optimizer, writer=writer, ckpt_dir=f"{path_out}/checkpoints", ckpt_name=f"model_{version}_best.pt",
        monitor="pr_auc", mode="max")


train 7/20:  10%|▉         | 292/3000 [00:06<00:58, 46.52it/s, loss=0.5896]

CPU times: user 2min 59s, sys: 44.8 s, total: 3min 44s
Wall time: 4min 58s


KeyboardInterrupt: 

In [ ]:
summary_train

{'best_score': 0.006281686572848036,
 'monitor': 'pr_auc',
 'mode': 'max',
 'epochs_ran': 20}

In [ ]:
trainer.metrics

{'train': {'accuracy': 0.9999825374906642,
  'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'roc_auc': nan,
  'pr_auc': nan,
  'loss': 0.0010990288452673116},
 'validate': {'accuracy': 0.992811,
  'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'roc_auc': 0.19263600100494416,
  'roc_curve': [array([0.00000000e+00, 1.00722077e-06, 1.78076631e-03, ...,
          9.99955682e-01, 9.99955682e-01, 1.00000000e+00]),
   array([0.        , 0.        , 0.        , ..., 0.99986051, 1.        ,
          1.        ])],
  'pr_auc': 0.004051092798500531,
  'precision_recall_curve': [array([0.007169  , 0.00716901, 0.00716901, ..., 0.        , 0.        ,
          1.        ]),
   array([1., 1., 1., ..., 0., 0., 0.]),
   0.007169],
  'loss': 0.00099914848303888},
 'best_model': {'best_score': 0.006281686572848036,
  'monitor': 'pr_auc',
  'mode': 'max',
  'epochs_ran': 20}}

In [ ]:
_ = trainer.evaluate(dataset_name="test")

test 0: 100%|██████████| 1000/1000 [00:09<00:00, 107.00it/s, loss=0.0010]


In [ ]:
trainer.metrics["test"]

{'accuracy': 0.992615,
 'precision': 0.0,
 'recall': 0.0,
 'f1': 0.0,
 'roc_auc': 0.18917966098046962,
 'roc_curve': [array([0.00000000e+00, 1.00741559e-06, 2.92049778e-03, ...,
         9.99973807e-01, 9.99973807e-01, 1.00000000e+00]),
  array([0.        , 0.        , 0.        , ..., 0.99986415, 1.        ,
         1.        ])],
 'pr_auc': 0.004148387107322318,
 'precision_recall_curve': [array([0.007361  , 0.00736101, 0.00736101, ..., 0.        , 0.        ,
         1.        ]),
  array([1., 1., 1., ..., 0., 0., 0.]),
  0.007361],
 'loss': 0.0010015654902672396}

In [ ]:
torch.save(model.state_dict(), f'{path_out}/model_{version}_model.pth')
with open(f'{path_out}/model_{version}_settings.yaml', "w") as f:
    yaml.safe_dump(dataset_train.config_file, f)

In [ ]:
safe_metrics = utils.make_json_safe(trainer.metrics)

with open(f'{path_out}/model_{version}_train_metrics.json', "w") as f:
    json.dump({model.__class__.__name__: safe_metrics}, f, indent=4)

In [ ]:
dataset_train.dataset.close()
writer.close()
utils.cleanup_workspace({})

###  Start TensorBoard

Run this in terminal:

tensorboard --logdir=\<path to tensor board log dir\> --host=0.0.0.0 --port=7007

Open:

http://localhost:7007/

Check effiency performance & memory use

for memory:
    
    from memory_profiler import profile

    @profile
    def your_function():

> python -m memory_profiler your_script.py  > mem_profile.txt

> python -m cProfile -s tottime your_script.py
> sudo py-spy top -- python your_script.py
 
> sudo py-spy record -o loader.svg -- python your_script.py